# Convolutional Neural Network

In [39]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm as tqdm

In [40]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [41]:
def weight(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
    
def bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [42]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## Placeholder

In [43]:
x  = tf.placeholder(tf.float32, shape=[None, 28*28])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

## First Convolutional Layer

In [44]:
# First Convolutional Layer
W_conv1 = weight([5, 5, 1, 32])
b_conv1 = bias([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1)

## Second Convolutional Layer

In [45]:
W_conv2 = weight([5, 5, 32, 64])
b_conv2 = bias([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2)

## Densely Connected Layer

In [46]:
W_fc1 = weight([7*7*64, 1024])
b_fc1 = bias([1024])

h_pool2_falt = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_falt, W_fc1) + b_fc1)

## Drop Layer

In [47]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer

In [48]:
W_fc2 = weight([1024, 10])
b_fc2 = bias([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## Deploy

In [49]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Train

In [51]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        print('epoch {}: accuracy {}'.format(i, accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

epoch 0: accuracy 0.11999999731779099
epoch 100: accuracy 0.8399999737739563
epoch 200: accuracy 0.9599999785423279
epoch 300: accuracy 0.9200000166893005
epoch 400: accuracy 0.9399999976158142
epoch 500: accuracy 0.9399999976158142
epoch 600: accuracy 0.9200000166893005
epoch 700: accuracy 0.9200000166893005
epoch 800: accuracy 0.9800000190734863
epoch 900: accuracy 0.9399999976158142
epoch 1000: accuracy 0.9800000190734863
epoch 1100: accuracy 0.9599999785423279
epoch 1200: accuracy 0.9800000190734863
epoch 1300: accuracy 0.9399999976158142
epoch 1400: accuracy 0.9599999785423279
epoch 1500: accuracy 0.9399999976158142
epoch 1600: accuracy 0.9599999785423279
epoch 1700: accuracy 0.9800000190734863
epoch 1800: accuracy 0.9599999785423279
epoch 1900: accuracy 0.9800000190734863


## Evaluation

In [52]:
print('Accuracy on test data:', accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1}))

Accuracy on test data: 0.975
